In [52]:
import cv2 as cv2
import tensorflow as tf
import numpy as np
from datetime import datetime

In [53]:
trainingBatches = np.load("dataBatch.npy") # Training Data
batchSize, height, width, channels = trainingBatches.shape # Shapes of training batches

# Some code to set up a log folder
present_time = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "logs"
logdir = "{}/run-{}/".format(root_logdir, present_time)

print(trainingBatches.shape)


(1200, 64, 64, 3)


In [54]:
tf.reset_default_graph()

# Build Phase
with tf.name_scope("CNN"):
    X = tf.placeholder(tf.float32, shape=(None, height, width, channels), name="Input") # Input batch of images
    conv1 = tf.layers.conv2d(inputs=X, filters=16, kernel_size=[8, 8], activation=tf.nn.relu, name="Conv1")
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2, name="Pool1")
    conv2 = tf.layers.conv2d(inputs=pool1, filters=16, kernel_size=[5, 5], activation=tf.nn.relu, name="Conv2")
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2, name="Pool2")
    pool2_flat = tf.layers.Flatten()(pool2)
    fc1 = tf.layers.dense(inputs=pool2_flat, units=256, activation=tf.nn.relu, name="FullyConnected1")
    feats = tf.layers.dense(inputs=fc1, units=16, name="Features")
    
with tf.name_scope("TripletLoss"): # Triplet loss function
    lossTriples = 0.0
    lossPairs = 0.0
    
    diff_pos = feats[0:batchSize:3] - feats[1:batchSize:3]
    diff_neg = feats[0:batchSize:3] - feats[2:batchSize:3]
    lossTriples = tf.reduce_sum(tf.maximum(0.0 , 1.0 - ((diff_neg * diff_neg) / (diff_pos * diff_pos + 0.01))))
    lossPairs = tf.reduce_sum(diff_pos * diff_pos)
   
    # Calculate Loss
    lossCombined = lossTriples + lossPairs
    
with tf.name_scope("Optimizer"): # Optimizer 
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    training_op = optimizer.minimize(lossCombined)
    
loss_summary = tf.summary.scalar('Triplet_Loss', lossCombined) # To log the losses
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph()) # Write the losses to a file

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [55]:
# Execution Phase

n_epochs = 1
batch_size = batchSize

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(batchSize):
            _, loss = sess.run([training_op, lossCombined], feed_dict = {X: trainingBatches})
            if (iteration % 50 == 0):
                summary_str = loss_summary.eval(feed_dict = {X: trainingBatches})
                step = (epoch + 1) * iteration
                file_writer.add_summary(summary_str, step)
                print("Epoch:", epoch, "| Step:", step, "| Loss:", loss)
                save_path = saver.save(sess, "./checkpoints/intermediate_model.ckpt")

file_writer.close()

Epoch: 0 | Step: 0 | Loss: 3061.54
Epoch: 0 | Step: 50 | Loss: 82.8111
Epoch: 0 | Step: 100 | Loss: 21.5896
Epoch: 0 | Step: 150 | Loss: 13.1138
Epoch: 0 | Step: 200 | Loss: 11.6786
Epoch: 0 | Step: 250 | Loss: 8.33506
Epoch: 0 | Step: 300 | Loss: 5.97838
Epoch: 0 | Step: 350 | Loss: 6.65972
Epoch: 0 | Step: 400 | Loss: 4.8618
Epoch: 0 | Step: 450 | Loss: 4.41915
Epoch: 0 | Step: 500 | Loss: 89.2198
Epoch: 0 | Step: 550 | Loss: 9.28268
Epoch: 0 | Step: 600 | Loss: 5.12985
Epoch: 0 | Step: 650 | Loss: 3.52739
Epoch: 0 | Step: 700 | Loss: 2.73131
Epoch: 0 | Step: 750 | Loss: 2.07223
Epoch: 0 | Step: 800 | Loss: 1.95864
Epoch: 0 | Step: 850 | Loss: 1.79926
Epoch: 0 | Step: 900 | Loss: 1.28874
Epoch: 0 | Step: 950 | Loss: 1.12925
Epoch: 0 | Step: 1000 | Loss: 1.16267
Epoch: 0 | Step: 1050 | Loss: 0.968565
Epoch: 0 | Step: 1100 | Loss: 0.937497
Epoch: 0 | Step: 1150 | Loss: 1.0711


In [56]:
db = np.load("database_images_data.npy")
test = np.load("test_images_data.npy")
db_images = []
test_images = []

for img in test[0]:
    test_images.append(img[0])

for img in db[0]:
    db_images.append(img[0])
    
test_images = np.asarray(test_images)
db_images = np.asarray(db_images)

In [57]:
print(test_images.shape)
print(db_images.shape)

(3540, 64, 64, 3)
(1335, 64, 64, 3)


In [58]:
with tf.Session() as sess:
    saver.restore(sess, "./checkpoints/intermediate_model.ckpt")
    dbDescriptor = feats.eval(feed_dict={X: db_images})
    testDescriptor = feats.eval(feed_dict={X: test_images})    
    
print(dbDescriptor.shape)
print(testDescriptor.shape)
    

INFO:tensorflow:Restoring parameters from ./checkpoints/intermediate_model.ckpt
(1335, 16)
(3540, 16)


In [59]:
dbDescriptorDatabase = []
testDescriptorDatabase = []

for index, testItem in enumerate(test[0]):
    label = testItem[1]
    descriptor = testDescriptor[index]
    testDescriptorDatabase.append([descriptor, label])
    
for index, dbItem in enumerate(db[0]):
    label = dbItem[1]
    descriptor = dbDescriptor[index]
    dbDescriptorDatabase.append([descriptor, label])
    
np.save("test_descriptors.npy", testDescriptorDatabase)
np.save("db_descriptors.npy", dbDescriptorDatabase)
